# Análisis y Entrenamiento del Detector de Peleas

Este notebook analiza y prepara los datos para el entrenamiento del modelo de detección de peleas. Incluye:
1. Exploración del dataset
2. Visualización de imágenes y distribuciones
3. Preparación de datos
4. Entrenamiento y evaluación del modelo

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import pandas as pd
from datetime import datetime

# Configurar estilo de las gráficas
plt.style.use('seaborn')
sns.set_palette('husl')

## 1. Exploración del Dataset

Analizaremos la estructura y distribución de nuestros datos de entrenamiento.

In [ ]:
def analyze_dataset(data_dir='data'):
    classes = ['NormalVideos', 'Fighting']
    stats = {}
    
    for class_name in classes:
        class_dir = os.path.join(data_dir, class_name)
        if os.path.exists(class_dir):
            # Contar imágenes
            images = [f for f in os.listdir(class_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
            stats[class_name] = {
                'count': len(images),
                'sample_images': images[:5] if images else []
            }
            
            # Analizar tamaños de imagen
            if images:
                sample_img = Image.open(os.path.join(class_dir, images[0]))
                stats[class_name]['image_size'] = sample_img.size
    
    return stats

dataset_stats = analyze_dataset()

# Mostrar estadísticas
for class_name, stats in dataset_stats.items():
    print(f"\nClase: {class_name}")
    print(f"Número de imágenes: {stats['count']}")
    if 'image_size' in stats:
        print(f"Tamaño de imagen: {stats['image_size']}")

### Visualización de la Distribución de Clases

In [ ]:
# Crear gráfico de distribución de clases
class_counts = [stats['count'] for stats in dataset_stats.values()]
class_names = list(dataset_stats.keys())

plt.figure(figsize=(10, 6))
sns.barplot(x=class_names, y=class_counts)
plt.title('Distribución de Clases en el Dataset')
plt.xlabel('Clase')
plt.ylabel('Número de Imágenes')
plt.xticks(rotation=45)
plt.show()

# Calcular y mostrar el ratio de desbalance
imbalance_ratio = max(class_counts) / min(class_counts)
print(f"\nRatio de desbalance: {imbalance_ratio:.2f}:1")

### Visualización de Imágenes de Muestra

In [ ]:
def plot_sample_images(data_dir='data', n_samples=5):
    classes = ['NormalVideos', 'Fighting']
    fig, axes = plt.subplots(2, n_samples, figsize=(15, 6))
    
    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        images = [f for f in os.listdir(class_dir) if f.endswith(('.jpg', '.jpeg', '.png'))][:n_samples]
        
        for j, img_name in enumerate(images):
            img_path = os.path.join(class_dir, img_name)
            img = Image.open(img_path)
            axes[i, j].imshow(img)
            axes[i, j].axis('off')
            if j == 0:
                axes[i, j].set_title(f'{class_name}\n{img.size}')
            else:
                axes[i, j].set_title(f'{img.size}')
    
    plt.tight_layout()
    plt.show()

plot_sample_images()

## 2. Preparación de Datos

Configuramos los generadores de datos con aumentación para el entrenamiento.

In [ ]:
from train_fight import Config, create_data_generators

# Mostrar configuración de aumentación de datos
print("Configuración de aumentación de datos:")
print(f"Rotación: ±{Config.ROTATION_RANGE}°")
print(f"Desplazamiento: ±{Config.SHIFT_RANGE * 100}%")
print(f"Zoom: ±{Config.ZOOM_RANGE * 100}%")
print(f"Rango de brillo: {Config.BRIGHTNESS_RANGE}")

# Crear generadores
train_generator, validation_generator, class_weights = create_data_generators()

print("\nPesos de clases para balance:")
for class_name, weight in zip(['NormalVideos', 'Fighting'], class_weights.values()):
    print(f"{class_name}: {weight:.2f}")

### Visualización de Aumentación de Datos

In [ ]:
def plot_augmented_samples(generator, n_samples=5):
    batch = next(generator)
    images, labels = batch
    
    fig, axes = plt.subplots(2, n_samples, figsize=(15, 6))
    class_names = ['Normal', 'Pelea']
    
    for i in range(2):
        class_images = images[labels == i][:n_samples]
        for j, img in enumerate(class_images):
            axes[i, j].imshow(img)
            axes[i, j].axis('off')
            if j == 0:
                axes[i, j].set_title(f'{class_names[i]}\n{img.shape}')
            else:
                axes[i, j].set_title(f'{img.shape}')
    
    plt.suptitle('Ejemplos de Imágenes Aumentadas')
    plt.tight_layout()
    plt.show()

plot_augmented_samples(train_generator)

## 3. Entrenamiento y Evaluación del Modelo

In [ ]:
from train_fight import create_model, Config

# Crear y entrenar modelo
model = create_model()
print("\nResumen del modelo:")
model.summary()

# Configurar callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=3,
        min_lr=Config.MIN_LR,
        verbose=1
    )
]

# Entrenar modelo
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=Config.EPOCHS,
    callbacks=callbacks,
    class_weight=class_weights
)

### Visualización de Métricas de Entrenamiento

In [ ]:
def plot_training_history(history):
    metrics = ['accuracy', 'precision', 'recall']
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    for i, metric in enumerate(metrics):
        axes[i].plot(history.history[metric], label='Train')
        axes[i].plot(history.history[f'val_{metric}'], label='Validation')
        axes[i].set_title(f'{metric.capitalize()} durante el Entrenamiento')
        axes[i].set_xlabel('Época')
        axes[i].set_ylabel(metric.capitalize())
        axes[i].legend()
    
    plt.tight_layout()
    plt.show()

plot_training_history(history)

### Evaluación Final del Modelo

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Evaluar modelo en conjunto de validación
y_pred = model.predict(validation_generator)
y_pred_classes = (y_pred > 0.5).astype(int)
y_true = validation_generator.classes

# Mostrar matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(
    confusion_matrix(y_true, y_pred_classes),
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=['Normal', 'Pelea'],
    yticklabels=['Normal', 'Pelea']
)
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

# Mostrar reporte de clasificación
print("\nReporte de Clasificación:")
print(classification_report(y_true, y_pred_classes, target_names=['Normal', 'Pelea']))